# CIS IEEE UNB - Renting Houce Prices Prediction

O desafio consiste em desenvolver um modelo capaz de prever o preço de imóveis do Rio de Janeiro cadastrados no AirBnB, tendo em vista as características do imóvel fornecidas durante o seu cadastro.


## Objetivos

1. Fazer uma análise do dataset utilizando ferramentas aprendidas no período
2. Demonstrar como pré-processar e representar os diferentes tipos de dados
3. Demonstrar a utilização de técnicas como one-hot encoding, redução de dimensionalidade e PCA;

    i.Como a redução da dimensionalidade afetou no desempenho do modelo?

4. Aplicar regressão nos dados - dividir o dataset em treinamento e
validação e aplicar, pelo menos, 3 algoritmos distintos de regressão para
prever o preço do imóvel. Testar visualizar os resultados, comparando o
desempenho dos modelos treinados;

    i. Qual modelo performou melhor?

5. Avaliar o desempenho do(s) modelo(s) treinado(s). A métrica da
competição será o erro quadrático médio (MSE), porém, recomenda-se a
avaliação do modelo por outras métricas adicionais, como o R2 e o erro
médio absoluto (MAE).

In [8]:
import pandas as pd

test_path = './data/test.csv'

test = pd.read_csv(test_path)

test.describe()

,id,scrape_id,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,number_of_reviews_ltm,calculated_host_listings_count_entire_homes,...,calculated_host_listings_count_shared_rooms,host_listings_count,latitude,longitude,accommodates,bathrooms,bedrooms,beds,minimum_nights,number_of_reviews
count,1.960300e+05,1.960300e+05,149877.000000,149877.000000,1.498770e+05,1.498770e+05,149877.000000,1.498770e+05,149877.000000,149877.000000,...,149877.000000,195939.000000,196030.000000,196030.000000,196030.000000,195663.000000,195847.000000,195455.000000,196030.000000,196030.000000
mean,1.691248e+07,2.019059e+13,4.511373,4.975447,2.762751e+04,2.763206e+04,4.624239,2.762944e+04,3.062798,6.216317,...,0.094711,8.834086,-22.965128,-43.253576,4.209585,1.695149,1.643681,2.600246,4.607295,8.048207
std,1.100082e+07,6.576807e+09,19.251823,19.792304,5.166299e+06,5.166299e+06,19.384673,5.166299e+06,7.586367,26.987452,...,0.765965,49.962856,0.034941,0.097347,2.575819,1.307152,1.085098,2.102016,19.898043,22.468177
min,1.787800e+04,2.018082e+13,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000,1.000000e+00,0.000000,0.000000,...,0.000000,0.000000,-23.073400,-43.737090,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,9.753521e+06,2.019012e+13,1.000000,1.000000,3.000000e+01,3.000000e+01,1.000000,3.000000e+01,0.000000,1.000000,...,0.000000,1.000000,-22.984660,-43.322160,2.000000,1.000000,1.000000,1.000000,1.000000,0.000000
50%,1.405750e+07,2.019062e+13,2.000000,3.000000,1.125000e+03,1.125000e+03,2.200000,1.125000e+03,0.000000,1.000000,...,0.000000,1.000000,-22.970640,-43.199830,4.000000,1.000000,1.000000,2.000000,2.000000,1.000000
75%,2.350016e+07,2.019122e+13,4.000000,5.000000,1.125000e+03,1.125000e+03,4.000000,1.125000e+03,2.000000,2.000000,...,0.000000,3.000000,-22.945361,-43.186863,6.000000,2.000000,2.000000,3.000000,4.000000,5.000000
max,4.352478e+07,2.020052e+13,1095.000000,1095.000000,1.000000e+09,1.000000e+09,1095.000000,1.000000e+09,122.000000,322.000000,...,23.000000,1495.000000,-22.750380,-43.104060,160.000000,200.000000,45.000000,69.000000,1224.000000,401.000000


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196030 entries, 0 to 196029
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   id                                            196030 non-null  int64  
 1   listing_url                                   196030 non-null  object 
 2   scrape_id                                     196030 non-null  float64
 3   last_scraped                                  196030 non-null  object 
 4   name                                          195706 non-null  object 
 5   summary                                       186334 non-null  object 
 6   space                                         118777 non-null  object 
 7   description                                   191715 non-null  object 
 8   experiences_offered                           196030 non-null  object 
 9   neighborhood_overview                         10